In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import SGD
import tensorflow as tf

更改處：  
1. 增加`import tensorflow as tf`
2. 增加model, 從3層改為4層
3. 更改model.fit 參數為10
4. 將result從model.predict.classses改為model.predict()
5. 增加classes_x = np.argmax()讓print(result[n])改為print(classes_x[n])以輸出數字
6. 把神經網路的compile optimizer從SGD改為tf.keras.optimizers.RMSprop
7. 把activation function 從ReLU改成ELU
8. 把n從9999改為779並執行

### 1. 讀入 MNSIT 數據集

In [3]:
from tensorflow.keras.datasets import mnist

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


### 2. 資料整理

### Channel

CNN 要注意一張圖有多少個 channel, 開始我們因為只有灰階, 所以只有一個 channel。因此我們要轉一下我們的資料格式:

    (28,28) --> (28, 28, 1)

In [5]:
x_train = x_train.reshape(60000, 28, 28, 1) / 255

In [6]:
x_test = x_test.reshape(10000, 28, 28, 1) / 255

In [7]:
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

### 3. step 1: 打造函數學習機 (CNN)

In [8]:
model = Sequential()

In [9]:
model.add(Conv2D(16, (3,3), padding='same',
                input_shape=(28,28,1),
                activation='relu'))

In [10]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [11]:
model.add(Conv2D(32, (3,3), padding='same',
                activation='relu'))

In [12]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [13]:
model.add(Conv2D(64, (3,3), padding='same',
                activation='relu'))

In [14]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [15]:
model.add(Conv2D(128, (3,3), padding='same',
                activation='relu'))

In [16]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [17]:
model.add(Flatten())

In [18]:
model.add(Dense(54, activation='relu'))

In [19]:
model.add(Dense(10, activation='softmax'))

#### 看一下我們的神經網路

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 64)         0

#### 組裝

In [31]:
model.compile(loss='mse', optimizer=tf.keras.optimizers.RMSprop(lr=0.087),
             metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


### 4. step 2: fit

In [32]:
model.fit(x_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
469/469 [==============================] - 13s 6ms/step - loss: 0.1801 - accuracy: 0.0987
Epoch 2/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.0987
Epoch 3/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.0987
Epoch 4/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.0987
Epoch 5/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.0987
Epoch 6/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.0987
Epoch 7/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.0987
Epoch 8/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.0987
Epoch 9/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.0987
Epoch 10/10
469/469 [==============================] - 2s 5ms/step - loss: 0.1803 - accuracy: 0.098

### Step 3. 預測

In [33]:
# result = model.predict_classes(x_test)
result = model.predict(x_test)
classes_x=np.argmax(result,axis=1)

### 看看測試資料表現如何

In [34]:
loss, acc = model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1804 - accuracy: 0.0980


In [35]:
print(f'測試資料的正確率為 {acc*100:.2f}%')

測試資料的正確率為 9.80%


In [36]:
def my_predict(n):
    # print('我可愛的 CNN 預測是', result[n])
    print('我可愛的 CNN 預測是', classes_x[n])
    X = x_test[n].reshape(28,28)
    plt.imshow(X, cmap='Greys')

In [37]:
from ipywidgets import interact_manual

In [38]:
interact_manual(my_predict, n=(0, 9999));

interactive(children=(IntSlider(value=4999, description='n', max=9999), Button(description='Run Interact', sty…

### 把我們的 model 存起來

In [39]:
model.save('myCNNmodel.h5')